In [1]:
!ls ../../dsc-phase-2-project-v2-3/data/kc_house_data.csv

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error as mae
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error


In [3]:
df = pd.read_csv("../../dsc-phase-2-project-v2-3/data/kc_house_data.csv", parse_dates=[1])
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [4]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,1788.596842,1970.999676,83.636778,98077.951845,47.560093,-122.213982,1986.620318,12758.283512
std,2.876736e+09,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,827.759761,29.375234,399.946414,53.513072,0.138552,0.140724,685.230472,27274.441950
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21597 non-null  int64         
 1   date           21597 non-null  datetime64[ns]
 2   price          21597 non-null  float64       
 3   bedrooms       21597 non-null  int64         
 4   bathrooms      21597 non-null  float64       
 5   sqft_living    21597 non-null  int64         
 6   sqft_lot       21597 non-null  int64         
 7   floors         21597 non-null  float64       
 8   waterfront     19221 non-null  object        
 9   view           21534 non-null  object        
 10  condition      21597 non-null  object        
 11  grade          21597 non-null  object        
 12  sqft_above     21597 non-null  int64         
 13  sqft_basement  21597 non-null  object        
 14  yr_built       21597 non-null  int64         
 15  yr_renovated   1775

In [6]:
df["zipcode"].value_counts()

98103    602
98038    589
98115    583
98052    574
98117    553
        ... 
98102    104
98010    100
98024     80
98148     57
98039     50
Name: zipcode, Length: 70, dtype: int64

In [7]:
df["waterfront"].value_counts()

NO     19075
YES      146
Name: waterfront, dtype: int64

In [8]:
df["view"].value_counts()

NONE         19422
AVERAGE        957
GOOD           508
FAIR           330
EXCELLENT      317
Name: view, dtype: int64

In [9]:
df["yr_renovated"].value_counts()

0.0       17011
2014.0       73
2003.0       31
2013.0       31
2007.0       30
          ...  
1946.0        1
1959.0        1
1971.0        1
1951.0        1
1954.0        1
Name: yr_renovated, Length: 70, dtype: int64

In [10]:
df["condition"].value_counts()

Average      14020
Good          5677
Very Good     1701
Fair           170
Poor            29
Name: condition, dtype: int64

In [11]:
df["grade"].value_counts()

7 Average        8974
8 Good           6065
9 Better         2615
6 Low Average    2038
10 Very Good     1134
11 Excellent      399
5 Fair            242
12 Luxury          89
4 Low              27
13 Mansion         13
3 Poor              1
Name: grade, dtype: int64

In [12]:
df["sqft_basement"].value_counts()

0.0       12826
?           454
600.0       217
500.0       209
700.0       208
          ...  
2580.0        1
1960.0        1
516.0         1
1913.0        1
1852.0        1
Name: sqft_basement, Length: 304, dtype: int64

In [13]:
map_cond = {
    "Poor" : -2,
    "Fair" : -1,
    "Average" : 0,
    "Good" : 1,
    "Very Good" : 2   
}

In [14]:
map_grade = {
    "3 Poor" : -4,
    "4 Low" : -3,
    "5 Fair" : -2,
    "6 Low Average" : -1,
    "7 Average" : 0,
    "8 Good" : 1,
    "9 Better" : 2,
    "10 Very Good" : 3,
    "11 Excellent" : 4,
    "12 Luxury" : 5,
    "13 Mansion" : 6
}

In [15]:
map_view = {
    "NONE" : -2,
    "FAIR" : -1,
    "AVERAGE" : 0,
    "GOOD" : 1,
    "EXCELLENT" : 2
}

In [16]:
map_water = {
    "NO" : 0,
    "YES" : 1
}

In [17]:
df["condition"] = df["condition"].map(map_cond)

In [18]:
df["grade"] = df["grade"].map(map_grade)

In [19]:
df["view"] = df["view"].map(map_view)

In [20]:
df["waterfront"] = df["waterfront"].map(map_water)

In [21]:
df_que = df.dropna()

In [22]:
df_que.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,1.576200e+04,1.576200e+04,15762.000000,15762.000000,15762.000000,1.576200e+04,15762.000000,15762.000000,15762.000000,15762.000000,15762.000000,15762.000000,15762.000000,15762.000000,15762.000000,15762.000000,15762.000000,15762.000000,15762.000000
mean,4.593364e+09,5.413172e+05,3.378949,2.120797,2084.512372,1.528082e+04,1.495147,0.007613,-1.770016,0.410862,0.663748,1792.775473,1971.111217,82.440236,98077.558241,47.559177,-122.213520,1990.219579,12900.415556
std,2.876078e+09,3.722258e+05,0.935301,0.766772,918.617686,4.182288e+04,0.539352,0.086924,0.761324,0.651961,1.172238,828.403502,29.336823,397.212626,53.414906,0.138629,0.140706,684.142495,27977.230059
min,1.000102e+06,8.200000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,-2.000000,-2.000000,-4.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,659.000000
25%,2.125159e+09,3.210000e+05,3.000000,1.750000,1430.000000,5.048500e+03,1.000000,0.000000,-2.000000,0.000000,0.000000,1200.000000,1952.000000,0.000000,98033.000000,47.469200,-122.328000,1490.000000,5100.000000
50%,3.905081e+09,4.500000e+05,3.000000,2.250000,1920.000000,7.602000e+03,1.500000,0.000000,-2.000000,0.000000,0.000000,1570.000000,1975.000000,0.000000,98065.000000,47.571000,-122.229000,1846.000000,7620.000000
75%,7.334501e+09,6.448750e+05,4.000000,2.500000,2550.000000,1.072000e+04,2.000000,0.000000,-2.000000,1.000000,1.000000,2220.000000,1997.000000,0.000000,98117.000000,47.677400,-122.124000,2370.000000,10107.500000
max,9.895000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,2.000000,2.000000,6.000000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [23]:
df_que = df_que.loc[df["sqft_basement"] != "?"]

In [24]:
df_que["sqft_basement"] = df_que["sqft_basement"].astype(float)

In [25]:
df_que = df_que.drop(columns=["date"])

In [26]:
# sns.set(rc = {'figure.figsize':(20,8)})
# sns.heatmap(df_que.corr(), annot=True)

In [27]:
X = df_que["sqft_living"]
y = df_que["price"]

sqft_price = sm.OLS(y, sm.add_constant(X)).fit()

In [28]:
simple_model_summary = sqft_price.summary()
simple_model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                 1.537e+04
Date:                Mon, 27 Jun 2022   Prob (F-statistic):               0.00
Time:                        17:10:04   Log-Likelihood:            -2.1450e+05
No. Observations:               15429   AIC:                         4.290e+05
Df Residuals:                   15427   BIC:                         4.290e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        -5.62e+04   5268.236    -10.668      0.000   -6.65e+04   -4.59e+04
sqft_living   286.5963      2.311    123.992      0.000     282.066     291.127
==============================================================================
Omnibus:                    10920.696   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           451159.757
Skew:                           2.916   Prob(JB):                         0.00
Kurtosis:                      28.841   Cond. No.                     5.65e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.65e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Just everything

In [29]:
X1 = df_que.drop(columns=["price"])
y = df_que["price"]

all_price = sm.OLS(y, sm.add_constant(X1)).fit()

In [30]:
simple_model_summary = all_price.summary()
simple_model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.701
Method:                 Least Squares   F-statistic:                     2009.
Date:                Mon, 27 Jun 2022   Prob (F-statistic):               0.00
Time:                        17:10:04   Log-Likelihood:            -2.1052e+05
No. Observations:               15429   AIC:                         4.211e+05
Df Residuals:                   15410   BIC:                         4.212e+05
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          6.465e+06   3.52e+06      1.835      0.067   -4.41e+05    1.34e+07
id            -1.087e-06    5.8e-07     -1.875      0.061   -2.22e-06    4.93e-08
bedrooms      -3.708e+04   2247.659    -16.498      0.000   -4.15e+04   -3.27e+04
bathrooms      4.373e+04   3910.981     11.181      0.000    3.61e+04    5.14e+04
sqft_living     115.6157      2.711     42.653      0.000     110.303     120.929
sqft_lot          0.0881      0.057      1.556      0.120      -0.023       0.199
floors         4561.2489   4330.220      1.053      0.292   -3926.493     1.3e+04
waterfront     6.191e+05   2.09e+04     29.607      0.000    5.78e+05     6.6e+05
view            5.36e+04   2587.912     20.711      0.000    4.85e+04    5.87e+04
condition       2.64e+04   2820.804      9.358      0.000    2.09e+04    3.19e+04
grade          9.382e+04   2603.243     36.039      0.000    8.87e+04    9.89e+04
sqft_above       76.5725      2.694     28.419      0.000      71.291      81.854
sqft_basement    39.0456      3.154     12.382      0.000      32.864      45.227
yr_built      -2680.1017     87.108    -30.768      0.000   -2850.844   -2509.360
yr_renovated     20.1897      4.434      4.553      0.000      11.499      28.881
zipcode        -569.8285     39.548    -14.408      0.000    -647.348    -492.309
lat             6.02e+05   1.29e+04     46.846      0.000    5.77e+05    6.27e+05
long          -2.147e+05   1.58e+04    -13.579      0.000   -2.46e+05   -1.84e+05
sqft_living15    16.6056      4.156      3.996      0.000       8.460      24.751
sqft_lot15       -0.3513      0.086     -4.071      0.000      -0.520      -0.182
==============================================================================
Omnibus:                    13395.001   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1472516.729
Skew:                           3.659   Prob(JB):                         0.00
Kurtosis:                      50.297   Cond. No.                     2.19e+22
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 9.46e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# Dropping based on heatmap

In [31]:
X2 = df_que.drop(columns=["price", "grade", "sqft_above", "sqft_living15", "bathrooms", "sqft_lot15"])
y = df_que["price"]

all_price_1 = sm.OLS(y, sm.add_constant(X2)).fit()

In [32]:
simple_model_summary = all_price_1.summary()
simple_model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     2207.
Date:                Mon, 27 Jun 2022   Prob (F-statistic):               0.00
Time:                        17:10:04   Log-Likelihood:            -2.1135e+05
No. Observations:               15429   AIC:                         4.227e+05
Df Residuals:                   15414   BIC:                         4.228e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          2.318e+06   3.64e+06      0.636      0.525   -4.82e+06    9.46e+06
id            -5.543e-07    6.1e-07     -0.909      0.364   -1.75e-06    6.42e-07
bedrooms      -4.341e+04   2305.476    -18.829      0.000   -4.79e+04   -3.89e+04
sqft_living     310.8034      3.068    101.306      0.000     304.790     316.817
sqft_lot         -0.0931      0.044     -2.128      0.033      -0.179      -0.007
floors         2.941e+04   4354.414      6.754      0.000    2.09e+04    3.79e+04
waterfront      5.81e+05    2.2e+04     26.364      0.000    5.38e+05    6.24e+05
view           6.877e+04   2676.200     25.695      0.000    6.35e+04     7.4e+04
condition       2.99e+04   2965.711     10.080      0.000    2.41e+04    3.57e+04
sqft_basement   -69.3700      5.268    -13.168      0.000     -79.696     -59.044
yr_built      -1525.2673     83.740    -18.214      0.000   -1689.407   -1361.127
yr_renovated     31.9597      4.627      6.908      0.000      22.891      41.028
zipcode        -697.5884     41.456    -16.827      0.000    -778.847    -616.330
lat            7.022e+05   1.33e+04     52.737      0.000    6.76e+05    7.28e+05
long          -2.932e+05   1.63e+04    -18.038      0.000   -3.25e+05   -2.61e+05
==============================================================================
Omnibus:                    11281.184   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           697024.558
Skew:                           2.925   Prob(JB):                         0.00
Kurtosis:                      35.404   Cond. No.                     1.14e+13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.14e+13. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Dropping based on p(statistic) and heatmap

In [33]:
X3 = df_que.drop(columns=["price", "grade", "sqft_above", "sqft_living15", "bathrooms", "sqft_lot15", "id"])
y = df_que["price"]

all_price_2 = sm.OLS(y, X3).fit()

In [34]:
simple_model_summary = all_price_2.summary()
simple_model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.893
Model:                            OLS   Adj. R-squared (uncentered):              0.893
Method:                 Least Squares   F-statistic:                              9886.
Date:                Mon, 27 Jun 2022   Prob (F-statistic):                        0.00
Time:                        17:10:04   Log-Likelihood:                     -2.1135e+05
No. Observations:               15429   AIC:                                  4.227e+05
Df Residuals:                   15416   BIC:                                  4.228e+05
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
bedrooms      -4.335e+04   2302.091    -18.831      0.000   -4.79e+04   -3.88e+04
sqft_living     311.1554      3.027    102.783      0.000     305.221     317.089
sqft_lot         -0.0850      0.043     -1.970      0.049      -0.170      -0.000
floors         2.877e+04   4250.813      6.769      0.000    2.04e+04    3.71e+04
waterfront     5.812e+05    2.2e+04     26.375      0.000    5.38e+05    6.24e+05
view           6.854e+04   2664.168     25.728      0.000    6.33e+04    7.38e+04
condition      3.022e+04   2931.880     10.308      0.000    2.45e+04     3.6e+04
sqft_basement   -70.0034      5.188    -13.492      0.000     -80.173     -59.833
yr_built      -1506.2073     77.790    -19.362      0.000   -1658.686   -1353.729
yr_renovated     32.1483      4.619      6.959      0.000      23.094      41.203
zipcode        -676.1093     21.941    -30.815      0.000    -719.116    -633.102
lat            7.022e+05   1.33e+04     52.760      0.000    6.76e+05    7.28e+05
long          -2.946e+05   1.62e+04    -18.195      0.000   -3.26e+05   -2.63e+05
==============================================================================
Omnibus:                    11274.899   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           694697.802
Skew:                           2.923   Prob(JB):                         0.00
Kurtosis:                      35.349   Cond. No.                     1.27e+06
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.27e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [35]:
# sns.pairplot(df_que.drop(columns=["grade", "sqft_above", "sqft_living15", "bathrooms", "sqft_lot15", "id"]))

In [36]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [37]:
lr_x1 = LinearRegression()

lr_x1.fit(X_train1, y_train1)

print(f"Train R2: {lr_x1.score(X_train1, y_train1):.4f}")
print(f"Test R2: {lr_x1.score(X_test1, y_test1):.4f}")

Train R2: 0.6978
Test R2: 0.7107


In [38]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [39]:
lr_x2 = LinearRegression()

lr_x2.fit(X_train2, y_train2)

print(f"Train R2: {lr_x2.score(X_train2, y_train2):.4f}")
print(f"Test R2: {lr_x2.score(X_test2, y_test2):.4f}")

Train R2: 0.6638
Test R2: 0.6768


In [40]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [41]:
lr_x3 = LinearRegression(fit_intercept=False)

lr_x3.fit(X_train3, y_train3)

print(f"Train R2: {lr_x3.score(X_train3, y_train3):.4f}")
print(f"Test R2: {lr_x3.score(X_test3, y_test3):.4f}")
reg = LinearRegression().fit(X_train3, y_train3)
print(f"R-Squares: {reg.score(X3, y)}%")
y_pred3 = reg.predict(X_test3)
print(f"RMSE: ${mae(y_test3, y_pred3, squared = False)}")
print(lr_x3.intercept_, lr_x3.coef_)

Train R2: 0.6637
Test R2: 0.6769
R-Squares: 0.6668542480767411%
RMSE: $205741.04499918557
0.0 [-4.41517906e+04  3.16260647e+02 -1.63519041e-01  2.92432858e+04
  5.95159918e+05  6.57713824e+04  2.95924013e+04 -6.66804271e+01
 -1.59218782e+03  3.91323652e+01 -6.78421045e+02  7.00584945e+05
 -2.98353524e+05]


In [42]:
# dict(df_que["zipcode"].value_counts()).keys()

In [43]:
df_pr_zip = df_que.drop(columns=["id", "bedrooms", "bathrooms", "sqft_living", "sqft_lot", "waterfront", "view", "condition", "grade", "sqft_above", "sqft_basement", "yr_built", "yr_renovated", "lat", "long", "sqft_living15", "sqft_lot15", "floors"]).groupby(df_que["zipcode"]).mean().head(71)

In [44]:
df_pr_zip = df_pr_zip.sort_values(by=['price'],ascending=True).drop(columns=["zipcode"])

In [45]:
df_pr_zip = df_pr_zip.reset_index().drop(columns=["price"])

In [46]:
map_zip = dict(dict(df_pr_zip)["zipcode"])

In [47]:
map_zip = {str(y):str(x) for x,y in map_zip.items()}

In [48]:
# map_zip

In [49]:
# df_que

In [50]:
df_que1 = df_que.copy()

In [51]:
df_que1["zipcode"] = df_que1["zipcode"].astype(str)

In [52]:
df_que1["zipcode"]

1        98125
3        98136
4        98074
5        98053
8        98146
         ...  
21591    98116
21592    98103
21593    98146
21594    98144
21596    98144
Name: zipcode, Length: 15429, dtype: object

In [53]:
df_que1["zipcode"] = df_que1["zipcode"].map(map_zip)

In [54]:
df_que1["zipcode"] = df_que1["zipcode"].astype(int)

In [55]:
# sns.set(rc = {'figure.figsize':(20,8)})
# sns.heatmap(df_que1.corr(), annot=True)

In [56]:
X4 = df_que1.drop(columns=["price", "grade", "sqft_above", "sqft_living15", "bathrooms", "sqft_lot15", "id"])
y1 = df_que1["price"]

all_price_4 = sm.OLS(y, X4).fit()

In [57]:
simple_model_summary = all_price_4.summary()
simple_model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.913
Model:                            OLS   Adj. R-squared (uncentered):              0.913
Method:                 Least Squares   F-statistic:                          1.245e+04
Date:                Mon, 27 Jun 2022   Prob (F-statistic):                        0.00
Time:                        17:10:05   Log-Likelihood:                     -2.0974e+05
No. Observations:               15429   AIC:                                  4.195e+05
Df Residuals:                   15416   BIC:                                  4.196e+05
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
bedrooms      -3.541e+04   2078.302    -17.037      0.000   -3.95e+04   -3.13e+04
sqft_living     269.1974      2.756     97.684      0.000     263.796     274.599
sqft_lot         -0.1402      0.038     -3.657      0.000      -0.215      -0.065
floors         1.097e+04   3824.227      2.868      0.004    3472.943    1.85e+04
waterfront      6.19e+05   1.99e+04     31.165      0.000     5.8e+05    6.58e+05
view           6.005e+04   2404.197     24.977      0.000    5.53e+04    6.48e+04
condition      1.518e+04   2647.064      5.736      0.000    9995.436    2.04e+04
sqft_basement   -55.9441      4.599    -12.163      0.000     -64.959     -46.929
yr_built      -1504.6764     67.514    -22.287      0.000   -1637.012   -1372.341
yr_renovated     18.0759      4.160      4.345      0.000       9.922      26.230
zipcode        8007.9867    116.326     68.841      0.000    7779.974    8235.999
lat           -1.145e+05   1.49e+04     -7.708      0.000   -1.44e+05   -8.54e+04
long          -6.819e+04   5801.572    -11.754      0.000   -7.96e+04   -5.68e+04
==============================================================================
Omnibus:                    14297.542   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1922934.215
Skew:                           4.021   Prob(JB):                         0.00
Kurtosis:                      57.097   Cond. No.                     5.69e+05
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 5.69e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [58]:
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y1, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [59]:
lr_x4 = LinearRegression()

lr_x4.fit(X_train4, y_train4)

print(f"Train R2: {lr_x4.score(X_train4, y_train4):.4f}")
print(f"Test R2: {lr_x4.score(X_test4, y_test4):.4f}")

Train R2: 0.7397
Test R2: 0.7503


In [60]:
# sns.pairplot(df_que1.drop(columns=["grade", "sqft_above", "sqft_living15", "bathrooms", "sqft_lot15", "id"]), kind='reg', plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.1}})

In [61]:
X_train5, X_test5, y_train5, y_test5 = train_test_split(X3, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [62]:
zip_train = X_train5[["zipcode"]]
zip_test = X_test5[["zipcode"]]
ohe = OneHotEncoder(handle_unknown='ignore',
                        sparse=False,
                        categories='auto')

ohe.fit(zip_train)


OneHotEncoder(handle_unknown='ignore', sparse=False)

In [63]:
zip_encoded_train = ohe.transform(zip_train)
zip_encoded_test = ohe.transform(zip_test)

In [64]:
zip_encoded_train = pd.DataFrame(
    # Pass in NumPy array
    zip_encoded_train,
    # Set the column names to the categories found by OHE
    columns=ohe.categories_[0],
    # Set the index to match X_train's index
    index=X_train5.index
)


In [65]:
zip_encoded_test = pd.DataFrame(
    # Pass in NumPy array
    zip_encoded_test,
    # Set the column names to the categories found by OHE
    columns=ohe.categories_[0],
    # Set the index to match X_train's index
    index=X_test5.index
)

In [66]:
X_train5.drop("zipcode", axis=1, inplace=True)

X_train5 = pd.concat([X_train5, zip_encoded_train], axis=1)


X_test5.drop("zipcode", axis=1, inplace=True)

X_test5 = pd.concat([X_test5, zip_encoded_test], axis=1)


C:\Users\AsuS\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [67]:
lr_x5 = LinearRegression()

lr_x5.fit(X_train5, y_train5)

print(f"Train R2: {lr_x5.score(X_train5, y_train5):.4f}")
print(f"Test R2: {lr_x5.score(X_test5, y_test5):.4f}")

Train R2: 0.7959
Test R2: 0.7962


In [68]:
all_price_5 = sm.OLS(y_test5, X_test5).fit()

In [69]:
simple_model_summary = all_price_5.summary()
simple_model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.806
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     194.2
Date:                Mon, 27 Jun 2022   Prob (F-statistic):               0.00
Time:                        17:10:05   Log-Likelihood:                -51686.
No. Observations:                3858   AIC:                         1.035e+05
Df Residuals:                    3776   BIC:                         1.040e+05
Df Model:                          81                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
bedrooms      -2.805e+04   3681.290     -7.620      0.000   -3.53e+04   -2.08e+04
sqft_living     275.7964      4.970     55.495      0.000     266.053     285.540
sqft_lot          0.2166      0.058      3.729      0.000       0.103       0.330
floors        -4.415e+04   7303.289     -6.045      0.000   -5.85e+04   -2.98e+04
waterfront      6.47e+05   2.99e+04     21.628      0.000    5.88e+05    7.06e+05
view           6.915e+04   4164.606     16.605      0.000     6.1e+04    7.73e+04
condition      3.069e+04   4546.743      6.749      0.000    2.18e+04    3.96e+04
sqft_basement  -117.0118      8.339    -14.033      0.000    -133.360    -100.663
yr_built        348.7973    140.328      2.486      0.013      73.672     623.923
yr_renovated     11.9675      7.045      1.699      0.089      -1.845      25.780
lat            2.321e+05   1.52e+05      1.527      0.127    -6.6e+04     5.3e+05
long          -2.118e+05   1.14e+05     -1.858      0.063   -4.35e+05    1.17e+04
98001         -3.752e+07   1.52e+07     -2.470      0.014   -6.73e+07   -7.74e+06
98002          -3.75e+07   1.52e+07     -2.470      0.014   -6.73e+07   -7.73e+06
98003         -3.754e+07   1.52e+07     -2.471      0.014   -6.73e+07   -7.75e+06
98004         -3.686e+07   1.52e+07     -2.424      0.015   -6.67e+07   -7.05e+06
98005         -3.718e+07   1.52e+07     -2.446      0.014    -6.7e+07   -7.38e+06
98006         -3.724e+07   1.52e+07     -2.451      0.014    -6.7e+07   -7.45e+06
98007         -3.727e+07   1.52e+07     -2.452      0.014   -6.71e+07   -7.47e+06
98008         -3.728e+07   1.52e+07     -2.454      0.014   -6.71e+07   -7.49e+06
98010         -3.738e+07   1.52e+07     -2.465      0.014   -6.71e+07   -7.65e+06
98011         -3.745e+07   1.52e+07     -2.462      0.014   -6.73e+07   -7.63e+06
98014         -3.741e+07   1.52e+07     -2.466      0.014   -6.72e+07   -7.67e+06
98019         -3.747e+07   1.52e+07     -2.468      0.014   -6.72e+07    -7.7e+06
98022         -3.742e+07   1.52e+07     -2.469      0.014   -6.71e+07   -7.71e+06
98023         -3.754e+07   1.52e+07     -2.470      0.014   -6.73e+07   -7.74e+06
98024         -3.735e+07   1.52e+07     -2.462      0.014   -6.71e+07   -7.61e+06
98027         -3.734e+07   1.52e+07     -2.460      0.014   -6.71e+07   -7.58e+06
98028         -3.747e+07   1.52e+07     -2.463      0.014   -6.73e+07   -7.64e+06
98029         -3.727e+07   1.52e+07     -2.456      0.014    -6.7e+07   -7.52e+06
98030         -3.749e+07   1.52e+07     -2.469      0.014   -6.73e+07   -7.72e+06
98031          -3.75e+07   1.52e+07     -2.469      0.014   -6.73e+07   -7.72e+06
98032         -3.755e+07   1.52e+07     -2.471      0.014   -6.73e+07   -7.76e+06
98033         -3.718e+07   1.52e+07     -2.445      0.015    -6.7e+07   -7.37e+06
98034         -3.738e+07   1.52e+07     -2.458   

In [70]:
X_train6, X_test6, y_train6, y_test6 = train_test_split(X3, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [71]:
Xy_train6 = pd.concat([X_train6, y_train6], axis=1)

Xy_train6_zip = Xy_train6.drop(columns=["zipcode", "bedrooms", "sqft_living", "sqft_lot", "waterfront", "view", "condition", "sqft_basement", "yr_built", "yr_renovated", "lat", "long", "floors"]).groupby(df_que["zipcode"]).mean().head(71)
Xy_train6_zip = Xy_train6_zip.sort_values(by=['price'],ascending=True).reset_index()
Xy_train6_zip = Xy_train6_zip.drop(columns=["price"])
map_zip1 = dict(dict(Xy_train6_zip)["zipcode"])
map_zip1 = {str(y):str(x) for x,y in map_zip1.items()}


In [77]:
# map_zip1

In [73]:
X_train6["zipcode"] = X_train6["zipcode"].astype(str)
X_train6["zipcode"] = X_train6["zipcode"].map(map_zip1)
X_train6["zipcode"] = X_train6["zipcode"].astype(int)

<ipython-input-73-3cef8f5147e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train6["zipcode"] = X_train6["zipcode"].astype(str)
<ipython-input-73-3cef8f5147e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train6["zipcode"] = X_train6["zipcode"].map(map_zip1)
<ipython-input-73-3cef8f5147e4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [75]:
X_test6["zipcode"] = X_test6["zipcode"].astype(str)
X_test6["zipcode"] = X_test6["zipcode"].map(map_zip1)
X_test6["zipcode"] = X_test6["zipcode"].astype(int)

<ipython-input-75-b981ac6cb024>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test6["zipcode"] = X_test6["zipcode"].astype(str)
<ipython-input-75-b981ac6cb024>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test6["zipcode"] = X_test6["zipcode"].map(map_zip1)
<ipython-input-75-b981ac6cb024>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [79]:
all_price_6 = sm.OLS(y_train6, X_train6).fit()
simple_model_summary = all_price_6.summary()
simple_model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.910
Model:                            OLS   Adj. R-squared (uncentered):              0.910
Method:                 Least Squares   F-statistic:                              9042.
Date:                Mon, 27 Jun 2022   Prob (F-statistic):                        0.00
Time:                        17:17:32   Log-Likelihood:                     -1.5748e+05
No. Observations:               11571   AIC:                                  3.150e+05
Df Residuals:                   11558   BIC:                                  3.151e+05
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
bedrooms      -3.754e+04   2407.912    -15.591      0.000   -4.23e+04   -3.28e+04
sqft_living     274.1687      3.234     84.786      0.000     267.830     280.507
sqft_lot         -0.2315      0.049     -4.771      0.000      -0.327      -0.136
floors           1.1e+04   4510.322      2.438      0.015    2157.156    1.98e+04
waterfront     6.224e+05   2.47e+04     25.226      0.000    5.74e+05    6.71e+05
view           5.793e+04   2818.312     20.555      0.000    5.24e+04    6.35e+04
condition      1.394e+04   3113.019      4.478      0.000    7838.712       2e+04
sqft_basement   -52.9279      5.384     -9.830      0.000     -63.482     -42.373
yr_built      -1564.4814     79.172    -19.761      0.000   -1719.672   -1409.291
yr_renovated     23.5465      4.887      4.818      0.000      13.967      33.126
zipcode        8143.1942    137.624     59.170      0.000    7873.427    8412.961
lat            -1.38e+05   1.76e+04     -7.845      0.000   -1.72e+05   -1.04e+05
long           -7.82e+04   6859.587    -11.400      0.000   -9.16e+04   -6.48e+04
==============================================================================
Omnibus:                    11323.665   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1762182.095
Skew:                           4.367   Prob(JB):                         0.00
Kurtosis:                      62.823   Cond. No.                     5.66e+05
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 5.66e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [80]:
lr_x6 = LinearRegression(fit_intercept=False)

lr_x6.fit(X_train6, y_train6)

print(f"Train R2: {lr_x6.score(X_train6, y_train6):.4f}")
print(f"Test R2: {lr_x6.score(X_test6, y_test6):.4f}")

Train R2: 0.7265
Test R2: 0.7358
